In [1]:
from probemapper.registration import ANTSPointTransformation, ANTSTransformationStages
from probemapper.io import save_nifti
import os, subprocess
import numpy as np
import pandas as pd

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220614_P201_laqueus_DiI_L-A11/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, f"indexing")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Transform channel coordinates

In [4]:
T = ANTSPointTransformation(
    ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin/",
    num_threads=8
)
T.set_transformations([
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_0GenericAffine.mat"), inverse=True),
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_1InverseWarp.nii.gz"), inverse=False),
])
T.set_moving_points(os.path.join(rootdir, "analysis/skeleton/channel_skeleton.csv"))
T.set_outname(os.path.join(tmpdir, "channel_LUT.csv"))

In [5]:
T.run()

['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsApplyTransformsToPoints', '-d', '3', '-i', '/bucket/ReiterU/DBS/data/20220614_P201_laqueus_DiI_L-A11/analysis/skeleton/channel_skeleton.csv', '-o', '/flash/ReiterU/tmp/220617_P201_skeletonization/indexing/channel_LUT.csv', '-t', '[/bucket/ReiterU/DBS/data/20220614_P201_laqueus_DiI_L-A11/analysis/ants/F2M_0GenericAffine.mat,1]', '-t', '/bucket/ReiterU/DBS/data/20220614_P201_laqueus_DiI_L-A11/analysis/ants/F2M_1InverseWarp.nii.gz']



### Query region ID

In [6]:
from probemapper import skeletonization

In [7]:
df = pd.read_csv(os.path.join(tmpdir, "channel_LUT.csv"))
coords = np.array([df["Z"], df["Y"], df["X"]])

In [8]:
regions = skeletonization.query_region_id2(coords,
                     "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                     100, coords.shape[1]-100, 10)

In [9]:
df["Region"] = regions

In [10]:
df.insert(0, "channel", df.pop("channel"))

In [11]:
display(df)

,channel,X,Y,Z,X_org,Y_org,Z_org,Region
0,-100,2185.73,3371.08,2261.25,1654.54,2372.01,1184.25,3
1,-99,2186.22,3362.60,2266.60,1654.85,2366.76,1193.95,3
2,-98,2186.67,3354.14,2271.85,1655.15,2361.51,1203.66,3
3,-97,2187.05,3345.56,2277.00,1655.46,2356.26,1213.36,3
4,-96,2187.50,3336.87,2282.04,1655.76,2351.01,1223.07,3
...,...,...,...,...,...,...,...,...
314,214,2475.47,1453.20,4227.42,1630.62,1320.45,4226.04,1
315,215,2474.57,1449.29,4236.42,1629.63,1320.27,4235.78,1
316,216,2473.54,1445.81,4245.51,1628.48,1320.59,4245.52,1
317,217,2472.30,1442.87,4254.64,1627.17,1321.44,4255.26,1


In [12]:
df.to_csv(os.path.join(tmpdir, "channel_LUT.csv"), index=False)

In [13]:
sk = skeletonization.generate_skeleton_image(coords[:, 100:],
                             "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                             10)

In [14]:
sk = sk.astype("uint8")
save_nifti(sk, os.path.join(tmpdir, "probe_image.nii.gz"), 1, 1, 1)

In [15]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/220617_P201_skeletonization/indexing', 'deigo:/bucket/ReiterU/DBS/data/20220614_P201_laqueus_DiI_L-A11/analysis'], returncode=0)